Set Up

In [ ]:
!pip install transformers datasets torch tqdm

import os
import torch
import pandas as pd
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from torch.utils.data import Dataset
from tqdm import tqdm
from google.colab import drive
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset
import pickle
from torch.optim import AdamW


# mount Google Drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/My Drive/Capstone"

# load mBERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

# load tokenized datasets to skip reprocessing
tokenized_datasets = {}
for split in ["train", "dev", "test"]:
    file_path = os.path.join(DATA_DIR, f"tokenized_{split}.pt")
    if os.path.exists(file_path):
        tokenized_datasets[split] = torch.load(file_path)
        print(f"✅ Loaded {split} dataset from {file_path}")
    else:
        print(f"❌ Missing file: {file_path}")

# dataset class for training
class CodeSwitchingDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.encodings["labels"][idx]
        }

# convert tokenized datasets to PyTorch Dataset
train_dataset = CodeSwitchingDataset(tokenized_datasets["train"])
dev_dataset = CodeSwitchingDataset(tokenized_datasets["dev"])
test_dataset = CodeSwitchingDataset(tokenized_datasets["test"])

# load pre-trained mBERT model with token classification head
num_labels = 6  # ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels
)

print("\n✅ Environment is set up. Ready to train!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

✅ Loaded train dataset from /content/drive/My Drive/Capstone/tokenized_train.pt
✅ Loaded dev dataset from /content/drive/My Drive/Capstone/tokenized_dev.pt
✅ Loaded test dataset from /content/drive/My Drive/Capstone/tokenized_test.pt


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Environment is set up. Ready to train!


Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = "/content/drive/My Drive/Capstone"
os.listdir(DATA_DIR)  # Should show 'train.conll'

['Capstone.ipynb',
 'dev.conll',
 'test.conll',
 'train.conll',
 'mBERT_model',
 'mBERT_trained',
 'tokenized_train.pt',
 'tokenized_dev.pt',
 'tokenized_test.pt',
 'mBERT_dev_results',
 'mBERT_test_results',
 'tokenized_dev_BETO.pt',
 'tokenized_test_BETO.pt',
 'tokenized_train_BETO.pt',
 'BETO_model',
 'BETO_results',
 'processed_train.pkl',
 'processed_dev.pkl',
 'processed_test.pkl',
 'RoBERTa_results',
 'RoBERTaTwitter_results',
 'mBERT_finetuned_gridsearch',
 'Internal Project Report - Capstone.gdoc',
 'Capstone_Finetuning.ipynb']

In [ ]:
!pip install transformers datasets torch

Load and Preprocess Datasets

In [ ]:
DATA_DIR = "/content/drive/My Drive/Capstone"

file_paths = {
    "train": os.path.join(DATA_DIR, "train.conll"),
    "dev": os.path.join(DATA_DIR, "dev.conll"),
    "test": os.path.join(DATA_DIR, "test.conll"),
}

# load CoNLL-formatted data
def load_conll_data(file_path):
    data = []
    sentence = []
    labels = []

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "":
                if sentence:  # store completed sentence
                    data.append((sentence, labels))
                    sentence = []
                    labels = []
            else:
                parts = line.split("\t")
                if len(parts) == 2:
                    token, label = parts
                    sentence.append(token)
                    labels.append(label)

    return data

# process and save all datasets
for split, path in file_paths.items():
    if os.path.exists(path):
        print(f"Processing {split} dataset...")
        dataset = load_conll_data(path)

        # convert to DataFrame
        df = pd.DataFrame(dataset, columns=["Tokens", "Labels"])

        # save processed data
        processed_path = os.path.join(DATA_DIR, f"processed_{split}.pkl")
        df.to_pickle(processed_path)
        print(f"Saved {split} dataset to: {processed_path}")
    else:
        print(f"File not found: {path}")

Processing train dataset...
Saved train dataset to: /content/drive/My Drive/Capstone/processed_train.pkl
Processing dev dataset...
Saved dev dataset to: /content/drive/My Drive/Capstone/processed_dev.pkl
Processing test dataset...
Saved test dataset to: /content/drive/My Drive/Capstone/processed_test.pkl


Preprocessing Verification

In [ ]:
DATA_DIR = "/content/drive/My Drive/Capstone"

for split in ["train", "dev", "test"]:
    processed_path = os.path.join(DATA_DIR, f"processed_{split}.pkl")

    if os.path.exists(processed_path):
        df = pd.read_pickle(processed_path)
        print(f"✅ {split} dataset loaded successfully! Sample:")
        print(df.head(), "\n")
    else:
        print(f"❌ Missing file: {processed_path}")

✅ train dataset loaded successfully! Sample:
                                              Tokens  \
0  [Aaaaaaaah, ., ¡¡, Pero, Willow, !!, El, 6x21,...   
1  [@SteffiGOES, Pues, no, lo, sabía, !, Pero, ha...   
2  [@anapaula_otero, Me, sale, invalid, address, ...   
3  [@quadritos, y, a, mi, por, qué, 2, veces, =(,...   
4  [justicia, !, ya, me, activaron, todas, las, n...   

                                              Labels  
0  [SPA, N, N, SPA, ENT, N, SPA, SPA, SPA, ENT, S...  
1  [N, SPA, SPA, SPA, SPA, N, SPA, SPA, SPA, SPA,...  
2  [N, SPA, SPA, ENG, ENG, N, SPA, SPA, SPA, SPA,...  
3  [N, SPA, SPA, SPA, SPA, SPA, N, SPA, N, SPA, N...  
4  [SPA, N, SPA, SPA, SPA, SPA, SPA, SPA, SPA, SP...   

✅ dev dataset loaded successfully! Sample:
                                              Tokens  \
0  [A, donde, quiera, que, vayas, ..., #nomaschav...   
1  [Currently, reading, 'CÓMO, CREAR, TU, PROPIA,...   
2  [Hoy, a, las, 22h, concierto, online, de, Love...   
3  [@dic7, Los, lib

mBERT Baseline model

Tokenization

In [ ]:
DATA_DIR = "/content/drive/My Drive/Capstone"

# load mBERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

# label mapping
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

print("Label Mapping:", label_to_id)

Label Mapping: {'SPA': 0, 'ENG': 1, 'BOR': 2, 'ENT': 3, 'OTH': 4, 'N': 5}


In [ ]:
def tokenize_and_align_labels(df):
    tokenized_inputs = []
    aligned_labels = []

    for tokens, labels in zip(df["Tokens"], df["Labels"]):
        tokenized_output = tokenizer(tokens, is_split_into_words=True, truncation=True, padding="max_length", max_length=256)

        # align labels to tokenized output
        word_ids = tokenized_output.word_ids()  # maps subwords to original words
        new_labels = []

        prev_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:  # padding token
                new_labels.append(-100)
            elif word_idx != prev_word_idx:  # first subword of a word
                new_labels.append(label_to_id[labels[word_idx]])
            else:  # subword of the same word
                new_labels.append(label_to_id[labels[word_idx]])

            prev_word_idx = word_idx

        tokenized_output["labels"] = new_labels
        tokenized_inputs.append(tokenized_output)

    return tokenized_inputs

In [ ]:
tokenized_datasets = {}

for split in ["train", "dev", "test"]:
    processed_path = os.path.join(DATA_DIR, f"processed_{split}.pkl")

    if os.path.exists(processed_path):
        print(f"Tokenizing {split} dataset...")

        # koad DataFrame
        df = pd.read_pickle(processed_path)

        # tokenize and align labels
        tokenized_data = tokenize_and_align_labels(df)

        # convert to PyTorch tensors
        tokenized_datasets[split] = {
            "input_ids": torch.tensor([d["input_ids"] for d in tokenized_data]),
            "attention_mask": torch.tensor([d["attention_mask"] for d in tokenized_data]),
            "labels": torch.tensor([d["labels"] for d in tokenized_data]),
        }

        # save tokenized dataset
        save_path = os.path.join(DATA_DIR, f"tokenized_{split}.pt")
        torch.save(tokenized_datasets[split], save_path)
        print(f"✅ {split} dataset saved to {save_path}")

    else:
        print(f"❌ Missing file: {processed_path}")

Tokenizing train dataset...
✅ train dataset saved to /content/drive/My Drive/Capstone/tokenized_train.pt
Tokenizing dev dataset...
✅ dev dataset saved to /content/drive/My Drive/Capstone/tokenized_dev.pt
Tokenizing test dataset...
✅ test dataset saved to /content/drive/My Drive/Capstone/tokenized_test.pt


Load Tokenized Data

In [ ]:
DATA_DIR = "/content/drive/My Drive/Capstone"

tokenized_datasets = {}
for split in ["train", "dev", "test"]:
    file_path = os.path.join(DATA_DIR, f"tokenized_{split}.pt")
    if os.path.exists(file_path):
        tokenized_datasets[split] = torch.load(file_path)
        print(f"✅ Loaded {split} dataset from {file_path}")
    else:
        print(f"❌ Missing file: {file_path}")

✅ Loaded train dataset from /content/drive/My Drive/Capstone/tokenized_train.pt
✅ Loaded dev dataset from /content/drive/My Drive/Capstone/tokenized_dev.pt
✅ Loaded test dataset from /content/drive/My Drive/Capstone/tokenized_test.pt


Prepare Data for Training

In [ ]:
class CodeSwitchingDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.encodings["labels"][idx]
        }

# convert tokenized datasets to PyTorch Dataset
train_dataset = CodeSwitchingDataset(tokenized_datasets["train"])
dev_dataset = CodeSwitchingDataset(tokenized_datasets["dev"])
test_dataset = CodeSwitchingDataset(tokenized_datasets["test"])

Load mBERT for Token Classification

In [ ]:
# # of labels
num_labels = 6  # ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]

#lLoad mBERT model with a token classification head
model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir=os.path.join(DATA_DIR, "mBERT_model"),  # save model in Google Drive
    evaluation_strategy="epoch",  # evaluate at the end of each epoch
    save_strategy="epoch",
    per_device_train_batch_size=8,  # adjust based on GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Define Trainer

In [ ]:
# data collator to handle padding dynamically during training
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-17-f6558305fbcd>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Train

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chloecrelia (chloecrelia-brandeis-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.126700,0.129025
2,0.082900,0.138797
3,0.040700,0.149072


TrainOutput(global_step=2847, training_loss=0.10597031695155207, metrics={'train_runtime': 424.6582, 'train_samples_per_second': 53.634, 'train_steps_per_second': 6.704, 'total_flos': 2975755076296704.0, 'train_loss': 0.10597031695155207, 'epoch': 3.0})

Save Trained Model

In [ ]:
model_path = os.path.join(DATA_DIR, "mBERT_trained")
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"✅ Model saved to {model_path}")

✅ Model saved to /content/drive/My Drive/Capstone/mBERT_trained


Load Trained Model

In [ ]:
model_path = os.path.join(DATA_DIR, "mBERT_trained")

model = BertForTokenClassification.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

print("✅ Model loaded successfully from Google Drive.")

✅ Model loaded successfully from Google Drive.


Prep Dev Set for Eval

In [ ]:
dev_dataset = CodeSwitchingDataset(tokenized_datasets["dev"])

dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False)

print("✅ Dev dataset ready for evaluation.")

✅ Dev dataset ready for evaluation.


Evaluate on Dev Set

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

model.eval()

true_labels_dev = []
pred_labels_dev = []

with torch.no_grad():
    for batch in tqdm(dev_dataloader, desc="Evaluating on Dev Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        # forward pass
        outputs = model(**batch)
        logits = outputs.logits  # get model predictions

        # convert logits to label predictions
        predictions = torch.argmax(logits, dim=-1)

        # store labels and predictions
        true_labels_dev.extend(batch["labels"].cpu().numpy())
        pred_labels_dev.extend(predictions.cpu().numpy())

print("✅ Inference on Dev Set complete.")

Evaluating on Dev Set: 100%|██████████| 120/120 [00:03<00:00, 36.50it/s]

✅ Inference on Dev Set complete.


In [ ]:
# label mapping dictionary again
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

print("✅ Label mapping dictionary restored.")

✅ Label mapping dictionary restored.


In [ ]:
true_labels_dev_flat = []
pred_labels_dev_flat = []

for true_seq, pred_seq in zip(true_labels_dev, pred_labels_dev):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:  # ignore padding tokens
            true_labels_dev_flat.append(id_to_label[true_label])
            pred_labels_dev_flat.append(id_to_label[pred_label])

# ✅ Debug
print(f"✅ Fixed: Length of true labels: {len(true_labels_dev_flat)}")
print(f"✅ Fixed: Length of predicted labels: {len(pred_labels_dev_flat)}")

✅ Fixed: Length of true labels: 31542
✅ Fixed: Length of predicted labels: 31542


In [ ]:
# accuracy
dev_accuracy = accuracy_score(true_labels_dev_flat, pred_labels_dev_flat)
print(f"🔹 **Dev Set Accuracy:** {dev_accuracy:.4f}")

# classification metrics (Precision, Recall, F1-score)
dev_report = classification_report(true_labels_dev_flat, pred_labels_dev_flat, target_names=label_list, digits=4)
print("\n🔹 **Dev Set Classification Report (Per Label):**\n", dev_report)

from sklearn.metrics import precision_recall_fscore_support

precision_dev, recall_dev, f1_dev, _ = precision_recall_fscore_support(true_labels_dev_flat, pred_labels_dev_flat, average="weighted")

print(f"\n🔹 **Overall Metrics (Dev Set):**")
print(f"   - Precision: {precision_dev:.4f}")
print(f"   - Recall: {recall_dev:.4f}")
print(f"   - F1-score: {f1_dev:.4f}")

🔹 **Dev Set Accuracy:** 0.9643

🔹 **Dev Set Classification Report (Per Label):**
               precision    recall  f1-score   support

         SPA     0.8174    0.8045    0.8109       445
         ENG     0.7944    0.8300    0.8118       894
         BOR     0.8564    0.8084    0.8317      2537
         ENT     0.9910    0.9968    0.9939      9145
         OTH     1.0000    0.2083    0.3448        48
           N     0.9771    0.9820    0.9795     18473

    accuracy                         0.9643     31542
   macro avg     0.9060    0.7717    0.7954     31542
weighted avg     0.9640    0.9643    0.9637     31542


🔹 **Overall Metrics (Dev Set):**
   - Precision: 0.9640
   - Recall: 0.9643
   - F1-score: 0.9637


Save Dev Results

In [ ]:
dev_results_path = os.path.join(DATA_DIR, "mBERT_dev_results", "dev_results.txt")

with open(dev_results_path, "w") as f:
    f.write(f"🔹 Dev Set Accuracy: {dev_accuracy:.4f}\n\n")
    f.write("🔹 **Overall Metrics (Dev Set):**\n")
    f.write(f"   - Precision: {precision_dev:.4f}\n")
    f.write(f"   - Recall: {recall_dev:.4f}\n")
    f.write(f"   - F1-score: {f1_dev:.4f}\n\n")
    f.write("🔹 **Dev Set Classification Report (Per Label):**\n")
    f.write(dev_report)

print(f"✅ Dev results saved to: {dev_results_path}")

NameError: name 'os' is not defined

Prep Test Set for Eval

In [ ]:
test_dataset = CodeSwitchingDataset(tokenized_datasets["test"])

test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print("✅ Test dataset ready for evaluation.")

✅ Test dataset ready for evaluation.


Evaluate on Test Set

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

model.eval()

true_labels_test = []
pred_labels_test = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating on Test Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        # forward pass
        outputs = model(**batch)
        logits = outputs.logits

        predictions = torch.argmax(logits, dim=-1)

        true_labels_test.extend(batch["labels"].cpu().numpy())
        pred_labels_test.extend(predictions.cpu().numpy())

print("✅ Inference on Test Set complete.")

Evaluating on Test Set: 100%|██████████| 119/119 [00:03<00:00, 37.12it/s]

✅ Inference on Test Set complete.


In [ ]:
true_labels_test_flat = []
pred_labels_test_flat = []

for true_seq, pred_seq in zip(true_labels_test, pred_labels_test):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_test_flat.append(id_to_label[true_label])
            pred_labels_test_flat.append(id_to_label[pred_label])

# ✅ Debug
print(f"✅ Fixed: Length of true labels: {len(true_labels_test_flat)}")
print(f"✅ Fixed: Length of predicted labels: {len(pred_labels_test_flat)}")

✅ Fixed: Length of true labels: 31261
✅ Fixed: Length of predicted labels: 31261


In [ ]:
test_accuracy = accuracy_score(true_labels_test_flat, pred_labels_test_flat)
print(f"🔹 **Test Set Accuracy:** {test_accuracy:.4f}")

test_report = classification_report(true_labels_test_flat, pred_labels_test_flat, target_names=label_list, digits=4)
print("\n🔹 **Test Set Classification Report (Per Label):**\n", test_report)

from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(true_labels_test_flat, pred_labels_test_flat, average="weighted")

print(f"\n🔹 **Overall Metrics:**")
print(f"   - Precision: {precision:.4f}")
print(f"   - Recall: {recall:.4f}")
print(f"   - F1-score: {f1:.4f}")

🔹 **Test Set Accuracy:** 0.9651

🔹 **Test Set Classification Report (Per Label):**
               precision    recall  f1-score   support

         SPA     0.7385    0.8179    0.7762       335
         ENG     0.8023    0.7524    0.7766      1030
         BOR     0.8653    0.8105    0.8370      2401
         ENT     0.9917    0.9955    0.9936      9104
         OTH     0.0000    0.0000    0.0000         6
           N     0.9777    0.9851    0.9814     18385

    accuracy                         0.9651     31261
   macro avg     0.7292    0.7269    0.7275     31261
weighted avg     0.9646    0.9651    0.9647     31261


🔹 **Overall Metrics:**
   - Precision: 0.9646
   - Recall: 0.9651
   - F1-score: 0.9647


Save Test Results

In [ ]:
test_results_path = os.path.join(DATA_DIR, "mBERT_test_results", "test_results.txt")

with open(test_results_path, "w") as f:
    f.write(f"🔹 Test Set Accuracy: {test_accuracy:.4f}\n\n")
    f.write("🔹 **Overall Metrics:**\n")
    f.write(f"   - Precision: {precision:.4f}\n")
    f.write(f"   - Recall: {recall:.4f}\n")
    f.write(f"   - F1-score: {f1:.4f}\n\n")
    f.write("🔹 **Test Set Classification Report (Per Label):**\n")
    f.write(test_report)

print(f"✅ Test results saved to: {test_results_path}")

✅ Test results saved to: /content/drive/My Drive/Capstone/mBERT_test_results/test_results.txt


BETO Baseline model

Load BETO Tokenizer

In [ ]:
# load BETO tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# label mappings
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

# tokenization + label alignment function
def tokenize_and_align_labels(examples):
    tokenized = tokenizer(
        examples["Tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    labels = []
    for i, label in enumerate(examples["Labels"]):
        word_ids = tokenized.word_ids(batch_index=i)
        label_ids = []
        prev_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(-100)
            prev_word_idx = word_idx
        labels.append(label_ids)

    tokenized["labels"] = labels
    return tokenized

Format datasets

In [ ]:
# load raw preprocessed .pkl files
with open(os.path.join(DATA_DIR, "processed_train.pkl"), "rb") as f:
    train_data = pickle.load(f)
with open(os.path.join(DATA_DIR, "processed_dev.pkl"), "rb") as f:
    dev_data = pickle.load(f)
with open(os.path.join(DATA_DIR, "processed_test.pkl"), "rb") as f:
    test_data = pickle.load(f)

# convert to HuggingFace Dataset format
train_dataset_raw = Dataset.from_dict(train_data)
dev_dataset_raw = Dataset.from_dict(dev_data)
test_dataset_raw = Dataset.from_dict(test_data)

# tokenize with alignment
train_dataset = train_dataset_raw.map(tokenize_and_align_labels, batched=True)
dev_dataset = dev_dataset_raw.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset_raw.map(tokenize_and_align_labels, batched=True)

train_dataset = train_dataset.remove_columns(["Tokens", "Labels"])
dev_dataset = dev_dataset.remove_columns(["Tokens", "Labels"])
test_dataset = test_dataset.remove_columns(["Tokens", "Labels"])

# format for PyTorch
train_dataset.set_format("torch")
dev_dataset.set_format("torch")
test_dataset.set_format("torch")

print("✅ Tokenized and cleaned datasets are ready for training.")

Map:   0%|          | 0/7592 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/949 [00:00<?, ? examples/s]

✅ Tokenized and cleaned datasets are ready for training.


Load BETO model

In [ ]:
# load BETO model with classification head
num_labels = len(label_list)
model = AutoModelForTokenClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased",
    num_labels=num_labels
)

print("✅ BETO model loaded.")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ BETO model loaded.


Train

In [ ]:
batch_size = 32
epochs = 3
learning_rate = 5e-5

data_collator = DataCollatorForTokenClassification(tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(epochs):
    print(f"🔁 Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_dataloader)
    print(f"✅ Avg loss: {avg_loss:.4f}")

🔁 Epoch 1/3


Training: 100%|██████████| 238/238 [01:08<00:00,  3.49it/s]


✅ Avg loss: 0.1791
🔁 Epoch 2/3


Training: 100%|██████████| 238/238 [01:07<00:00,  3.52it/s]


✅ Avg loss: 0.0768
🔁 Epoch 3/3


Training: 100%|██████████| 238/238 [01:07<00:00,  3.52it/s]

✅ Avg loss: 0.0419


Save Trained Model

In [ ]:
beto_model_dir = os.path.join(DATA_DIR, "BETO_model")
model.save_pretrained(beto_model_dir)
tokenizer.save_pretrained(beto_model_dir)
print(f"✅ BETO model saved to {beto_model_dir}")

✅ BETO model saved to /content/drive/My Drive/Capstone/BETO_model


Evaluate on Dev

In [ ]:
dev_dataloader = DataLoader(dev_dataset, batch_size=32, collate_fn=data_collator)

model.eval()

true_labels_dev = []
pred_labels_dev = []

device = "cuda" if torch.cuda.is_available() else "cpu"

with torch.no_grad():
    for batch in tqdm(dev_dataloader, desc="Evaluating on Dev Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels_dev.extend(batch["labels"].cpu().numpy())
        pred_labels_dev.extend(predictions.cpu().numpy())

print("✅ Inference on Dev Set complete.")


Evaluating on Dev Set: 100%|██████████| 30/30 [00:03<00:00,  9.85it/s]

✅ Inference on Dev Set complete.


map to labels

In [ ]:
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
id_to_label = {i: label for i, label in enumerate(label_list)}

# flatten and convert to label names, aligned properly
true_labels_dev_flat = []
pred_labels_dev_flat = []

for true_seq, pred_seq in zip(true_labels_dev, pred_labels_dev):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_dev_flat.append(id_to_label[true_label])
            pred_labels_dev_flat.append(id_to_label[pred_label])

print("✅ Dev labels aligned, filtered, and converted to text format.")

✅ Dev labels aligned, filtered, and converted to text format.


Print and save BETO dev results

In [ ]:
dev_accuracy = accuracy_score(true_labels_dev_flat, pred_labels_dev_flat)
dev_report = classification_report(true_labels_dev_flat, pred_labels_dev_flat, target_names=label_list, digits=4)

print(f"🔹 Dev Set Accuracy: {dev_accuracy:.4f}")
print("\n🔹 Dev Set Classification Report:\n", dev_report)

import os
os.makedirs(os.path.join(DATA_DIR, "BETO_results"), exist_ok=True)
dev_results_path = os.path.join(DATA_DIR, "BETO_results", "dev_results.txt")

with open(dev_results_path, "w") as f:
    f.write(f"Dev Set Accuracy: {dev_accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(dev_report)

print(f"✅ Dev results saved to {dev_results_path}")

🔹 Dev Set Accuracy: 0.9688

🔹 Dev Set Classification Report:
               precision    recall  f1-score   support

         SPA     0.8594    0.7483    0.8000       294
         ENG     0.8637    0.8035    0.8325       631
         BOR     0.8355    0.8807    0.8575      1609
         ENT     0.9903    0.9946    0.9925      3917
         OTH     0.8333    0.3571    0.5000        28
           N     0.9862    0.9858    0.9860     13387

    accuracy                         0.9688     19866
   macro avg     0.8948    0.7950    0.8281     19866
weighted avg     0.9689    0.9688    0.9686     19866

✅ Dev results saved to /content/drive/My Drive/Capstone/BETO_results/dev_results.txt


Evaluate on test set

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

model.eval()

true_labels_test = []
pred_labels_test = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating on Test Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels_test.extend(batch["labels"].cpu().numpy())
        pred_labels_test.extend(predictions.cpu().numpy())

print("✅ Inference on Test Set complete.")

Evaluating on Test Set: 100%|██████████| 30/30 [00:03<00:00,  9.79it/s]

✅ Inference on Test Set complete.


map to labels

In [ ]:
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
id_to_label = {i: label for i, label in enumerate(label_list)}

true_labels_test_flat = []
pred_labels_test_flat = []

for true_seq, pred_seq in zip(true_labels_test, pred_labels_test):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_test_flat.append(id_to_label[true_label])
            pred_labels_test_flat.append(id_to_label[pred_label])

print("✅ Test labels aligned and converted to text format.")

✅ Test labels aligned and converted to text format.


Print and save BETO test results

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

test_accuracy = accuracy_score(true_labels_test_flat, pred_labels_test_flat)
test_report = classification_report(true_labels_test_flat, pred_labels_test_flat, target_names=label_list, digits=4)

print(f"🔹 Test Set Accuracy: {test_accuracy:.4f}")
print("\n🔹 Test Set Classification Report:\n", test_report)

test_results_path = os.path.join(DATA_DIR, "BETO_results", "test_results.txt")

with open(test_results_path, "w") as f:
    f.write(f"Test Set Accuracy: {test_accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(test_report)

print(f"✅ Test results saved to {test_results_path}")

🔹 Test Set Accuracy: 0.9660

🔹 Test Set Classification Report:
               precision    recall  f1-score   support

         SPA     0.8052    0.7500    0.7766       248
         ENG     0.8467    0.7115    0.7732       714
         BOR     0.8267    0.8657    0.8457      1504
         ENT     0.9911    0.9939    0.9925      3911
         OTH     0.1429    0.5000    0.2222         4
           N     0.9839    0.9868    0.9854     13468

    accuracy                         0.9660     19849
   macro avg     0.7661    0.8013    0.7659     19849
weighted avg     0.9661    0.9660    0.9658     19849

✅ Test results saved to /content/drive/My Drive/Capstone/BETO_results/test_results.txt


RoBERTa Baseline Model

Tokenize and prepare

In [ ]:
# load RoBERTa-BNE tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "PlanTL-GOB-ES/roberta-base-bne",
    add_prefix_space=True  # ✅ important for RoBERTa with split tokens
)

# define labels
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

# tokenization + label alignment function
def tokenize_and_align_labels(examples):
    tokenized = tokenizer(
        examples["Tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    labels = []
    for i, label in enumerate(examples["Labels"]):
        word_ids = tokenized.word_ids(batch_index=i)
        label_ids = []
        prev_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(-100)
            prev_word_idx = word_idx
        labels.append(label_ids)

    tokenized["labels"] = labels
    return tokenized

In [ ]:
# load preprocessed .pkl files
with open(os.path.join(DATA_DIR, "processed_train.pkl"), "rb") as f:
    train_data = pickle.load(f)
with open(os.path.join(DATA_DIR, "processed_dev.pkl"), "rb") as f:
    dev_data = pickle.load(f)
with open(os.path.join(DATA_DIR, "processed_test.pkl"), "rb") as f:
    test_data = pickle.load(f)

train_dataset_raw = Dataset.from_dict(train_data)
dev_dataset_raw = Dataset.from_dict(dev_data)
test_dataset_raw = Dataset.from_dict(test_data)

train_dataset = train_dataset_raw.map(tokenize_and_align_labels, batched=True)
dev_dataset = dev_dataset_raw.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset_raw.map(tokenize_and_align_labels, batched=True)

train_dataset = train_dataset.remove_columns(["Tokens", "Labels"])
dev_dataset = dev_dataset.remove_columns(["Tokens", "Labels"])
test_dataset = test_dataset.remove_columns(["Tokens", "Labels"])

train_dataset.set_format("torch")
dev_dataset.set_format("torch")
test_dataset.set_format("torch")

print("✅ RoBERTa-BNE tokenized datasets ready.")

Map:   0%|          | 0/7592 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/949 [00:00<?, ? examples/s]

✅ RoBERTa-BNE tokenized datasets ready.


Load roberta model

In [ ]:
num_labels = len(label_list)

model = AutoModelForTokenClassification.from_pretrained(
    "PlanTL-GOB-ES/roberta-base-bne",
    num_labels=num_labels
)

print("✅ RoBERTa-BNE model loaded.")

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ RoBERTa-BNE model loaded.


Train

In [ ]:
# training parameters
batch_size = 32
epochs = 3
learning_rate = 5e-5

data_collator = DataCollatorForTokenClassification(tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(epochs):
    print(f"🔁 Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_dataloader)
    print(f"✅ Avg loss for epoch {epoch + 1}: {avg_loss:.4f}")

🔁 Epoch 1/3


Training: 100%|██████████| 238/238 [01:07<00:00,  3.51it/s]


✅ Avg loss for epoch 1: 0.1649
🔁 Epoch 2/3


Training: 100%|██████████| 238/238 [01:06<00:00,  3.56it/s]


✅ Avg loss for epoch 2: 0.0603
🔁 Epoch 3/3


Training: 100%|██████████| 238/238 [01:06<00:00,  3.56it/s]

✅ Avg loss for epoch 3: 0.0275


Save trained roberta

In [ ]:
roberta_save_path = os.path.join(DATA_DIR, "RoBERTa_results", "model")
os.makedirs(roberta_save_path, exist_ok=True)

model.save_pretrained(roberta_save_path)
tokenizer.save_pretrained(roberta_save_path)

print(f"✅ Trained RoBERTa-BNE model saved to: {roberta_save_path}")

✅ Trained RoBERTa-BNE model saved to: /content/drive/My Drive/Capstone/RoBERTa_results/model


Load saved model and tokenizer

In [ ]:
roberta_save_path = os.path.join(DATA_DIR, "RoBERTa_results", "model")

model = AutoModelForTokenClassification.from_pretrained(roberta_save_path)
tokenizer = AutoTokenizer.from_pretrained(roberta_save_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("✅ Loaded saved RoBERTa-BNE model for evaluation.")

Evaluate on Dev

In [ ]:
dev_dataloader = DataLoader(dev_dataset, batch_size=32, collate_fn=data_collator)

model.eval()

true_labels_dev = []
pred_labels_dev = []

with torch.no_grad():
    for batch in tqdm(dev_dataloader, desc="Evaluating on Dev Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels_dev.extend(batch["labels"].cpu().numpy())
        pred_labels_dev.extend(predictions.cpu().numpy())

print("✅ Inference on Dev Set complete.")

Evaluating on Dev Set: 100%|██████████| 30/30 [00:02<00:00, 10.04it/s]

✅ Inference on Dev Set complete.


In [ ]:
label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
id_to_label = {i: label for i, label in enumerate(label_list)}

true_labels_dev_flat = []
pred_labels_dev_flat = []

for true_seq, pred_seq in zip(true_labels_dev, pred_labels_dev):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_dev_flat.append(id_to_label[true_label])
            pred_labels_dev_flat.append(id_to_label[pred_label])

print("✅ Dev labels aligned and converted.")

✅ Dev labels aligned and converted.


In [ ]:
dev_accuracy = accuracy_score(true_labels_dev_flat, pred_labels_dev_flat)
dev_report = classification_report(true_labels_dev_flat, pred_labels_dev_flat, target_names=label_list, digits=4)

print(f"🔹 Dev Accuracy: {dev_accuracy:.4f}")
print("\n🔹 Dev Classification Report:\n", dev_report)

os.makedirs(os.path.join(DATA_DIR, "RoBERTa_results"), exist_ok=True)
dev_results_path = os.path.join(DATA_DIR, "RoBERTa_results", "dev_results.txt")

with open(dev_results_path, "w") as f:
    f.write(f"Dev Accuracy: {dev_accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(dev_report)

print(f"✅ Dev results saved to {dev_results_path}")

🔹 Dev Accuracy: 0.9724

🔹 Dev Classification Report:
               precision    recall  f1-score   support

         SPA     0.7596    0.8707    0.8114       294
         ENG     0.8381    0.8447    0.8414       631
         BOR     0.9035    0.8496    0.8757      1609
         ENT     0.9974    0.9951    0.9963      3917
         OTH     0.8889    0.2857    0.4324        28
           N     0.9846    0.9901    0.9873     13387

    accuracy                         0.9724     19866
   macro avg     0.8953    0.8060    0.8241     19866
weighted avg     0.9724    0.9724    0.9720     19866

✅ Dev results saved to /content/drive/My Drive/Capstone/RoBERTa_results/dev_results.txt


Evaluate on Test

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

true_labels_test = []
pred_labels_test = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating on Test Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels_test.extend(batch["labels"].cpu().numpy())
        pred_labels_test.extend(predictions.cpu().numpy())

print("✅ Inference on Test Set complete.")

Evaluating on Test Set: 100%|██████████| 30/30 [00:02<00:00, 10.06it/s]

✅ Inference on Test Set complete.


In [ ]:
true_labels_test_flat = []
pred_labels_test_flat = []

for true_seq, pred_seq in zip(true_labels_test, pred_labels_test):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_test_flat.append(id_to_label[true_label])
            pred_labels_test_flat.append(id_to_label[pred_label])

print("✅ Test labels aligned and converted.")

✅ Test labels aligned and converted.


In [ ]:
test_accuracy = accuracy_score(true_labels_test_flat, pred_labels_test_flat)
test_report = classification_report(true_labels_test_flat, pred_labels_test_flat, target_names=label_list, digits=4)

print(f"🔹 Test Accuracy: {test_accuracy:.4f}")
print("\n🔹 Test Classification Report:\n", test_report)

test_results_path = os.path.join(DATA_DIR, "RoBERTa_results", "test_results.txt")

with open(test_results_path, "w") as f:
    f.write(f"Test Accuracy: {test_accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(test_report)

print(f"✅ Test results saved to {test_results_path}")

🔹 Test Accuracy: 0.9721

🔹 Test Classification Report:
               precision    recall  f1-score   support

         SPA     0.6935    0.8669    0.7706       248
         ENG     0.8285    0.7983    0.8131       714
         BOR     0.9072    0.8251    0.8642      1504
         ENT     0.9985    0.9954    0.9969      3911
         OTH     0.0000    0.0000    0.0000         4
           N     0.9851    0.9932    0.9891     13468

    accuracy                         0.9721     19849
   macro avg     0.7355    0.7465    0.7390     19849
weighted avg     0.9723    0.9721    0.9719     19849

✅ Test results saved to /content/drive/My Drive/Capstone/RoBERTa_results/test_results.txt


RoBERTa Twitter Baseline Model

Tokenizing

In [ ]:
# load RoBERTa-Twitter tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "cardiffnlp/twitter-roberta-base",
    add_prefix_space=True  # required for RoBERTa + is_split_into_words
)

label_list = ["SPA", "ENG", "BOR", "ENT", "OTH", "N"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

# tokenization + label alignment function
def tokenize_and_align_labels(examples):
    tokenized = tokenizer(
        examples["Tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    labels = []
    for i, label in enumerate(examples["Labels"]):
        word_ids = tokenized.word_ids(batch_index=i)
        label_ids = []
        prev_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(-100)
            prev_word_idx = word_idx
        labels.append(label_ids)

    tokenized["labels"] = labels
    return tokenized

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
with open(os.path.join(DATA_DIR, "processed_train.pkl"), "rb") as f:
    train_data = pickle.load(f)
with open(os.path.join(DATA_DIR, "processed_dev.pkl"), "rb") as f:
    dev_data = pickle.load(f)
with open(os.path.join(DATA_DIR, "processed_test.pkl"), "rb") as f:
    test_data = pickle.load(f)

train_dataset_raw = Dataset.from_dict(train_data)
dev_dataset_raw = Dataset.from_dict(dev_data)
test_dataset_raw = Dataset.from_dict(test_data)

train_dataset = train_dataset_raw.map(tokenize_and_align_labels, batched=True)
dev_dataset = dev_dataset_raw.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset_raw.map(tokenize_and_align_labels, batched=True)

train_dataset = train_dataset.remove_columns(["Tokens", "Labels"])
dev_dataset = dev_dataset.remove_columns(["Tokens", "Labels"])
test_dataset = test_dataset.remove_columns(["Tokens", "Labels"])

train_dataset.set_format("torch")
dev_dataset.set_format("torch")
test_dataset.set_format("torch")

print("✅ RoBERTa-Twitter datasets tokenized and ready.")

Map:   0%|          | 0/7592 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/949 [00:00<?, ? examples/s]

✅ RoBERTa-Twitter datasets tokenized and ready.


Load roberta twitter model

In [ ]:
num_labels = len(label_list)

model = AutoModelForTokenClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base",
    num_labels=num_labels
)

print("✅ RoBERTa-Twitter model loaded.")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ RoBERTa-Twitter model loaded.


Train

In [ ]:
batch_size = 32
epochs = 3
learning_rate = 5e-5

data_collator = DataCollatorForTokenClassification(tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(epochs):
    print(f"🔁 Epoch {epoch + 1}/{epochs}")
    total_loss = 0

    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_dataloader)
    print(f"✅ Avg loss for epoch {epoch + 1}: {avg_loss:.4f}")

🔁 Epoch 1/3


Training: 100%|██████████| 238/238 [01:08<00:00,  3.47it/s]


✅ Avg loss for epoch 1: 0.1890
🔁 Epoch 2/3


Training: 100%|██████████| 238/238 [01:07<00:00,  3.51it/s]


✅ Avg loss for epoch 2: 0.0892
🔁 Epoch 3/3


Training: 100%|██████████| 238/238 [01:07<00:00,  3.51it/s]

✅ Avg loss for epoch 3: 0.0577


Save trained model

In [ ]:
roberta_twitter_save_path = os.path.join(DATA_DIR, "RoBERTaTwitter_results", "model")
os.makedirs(roberta_twitter_save_path, exist_ok=True)

model.save_pretrained(roberta_twitter_save_path)
tokenizer.save_pretrained(roberta_twitter_save_path)

print(f"✅ Trained RoBERTa-Twitter model saved to: {roberta_twitter_save_path}")

✅ Trained RoBERTa-Twitter model saved to: /content/drive/My Drive/Capstone/RoBERTaTwitter_results/model


Evaluate on dev

In [ ]:
dev_dataloader = DataLoader(dev_dataset, batch_size=32, collate_fn=data_collator)

model.eval()

true_labels_dev = []
pred_labels_dev = []

with torch.no_grad():
    for batch in tqdm(dev_dataloader, desc="Evaluating on Dev Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels_dev.extend(batch["labels"].cpu().numpy())
        pred_labels_dev.extend(predictions.cpu().numpy())

print("✅ Inference on Dev Set complete.")

Evaluating on Dev Set: 100%|██████████| 30/30 [00:02<00:00, 10.06it/s]

✅ Inference on Dev Set complete.


In [ ]:
id_to_label = {i: label for i, label in enumerate(label_list)}

true_labels_dev_flat = []
pred_labels_dev_flat = []

for true_seq, pred_seq in zip(true_labels_dev, pred_labels_dev):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_dev_flat.append(id_to_label[true_label])
            pred_labels_dev_flat.append(id_to_label[pred_label])

print("✅ Dev labels flattened and aligned.")

✅ Dev labels flattened and aligned.


In [ ]:
dev_accuracy = accuracy_score(true_labels_dev_flat, pred_labels_dev_flat)
dev_report = classification_report(true_labels_dev_flat, pred_labels_dev_flat, target_names=label_list, digits=4)

print(f"🔹 Dev Accuracy: {dev_accuracy:.4f}")
print("\n🔹 Dev Classification Report:\n", dev_report)

os.makedirs(os.path.join(DATA_DIR, "RoBERTaTwitter_results"), exist_ok=True)
dev_results_path = os.path.join(DATA_DIR, "RoBERTaTwitter_results", "dev_results.txt")

with open(dev_results_path, "w") as f:
    f.write(f"Dev Accuracy: {dev_accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(dev_report)

print(f"✅ Dev results saved to {dev_results_path}")

🔹 Dev Accuracy: 0.9708

🔹 Dev Classification Report:
               precision    recall  f1-score   support

         SPA     0.8368    0.8197    0.8282       294
         ENG     0.8392    0.8685    0.8536       631
         BOR     0.8907    0.8303    0.8594      1609
         ENT     0.9957    0.9977    0.9967      3917
         OTH     0.7500    0.1071    0.1875        28
           N     0.9818    0.9898    0.9858     13387

    accuracy                         0.9708     19866
   macro avg     0.8824    0.7689    0.7852     19866
weighted avg     0.9701    0.9708    0.9700     19866

✅ Dev results saved to /content/drive/My Drive/Capstone/RoBERTaTwitter_results/dev_results.txt


Evaluate on test

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

true_labels_test = []
pred_labels_test = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating on Test Set"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels_test.extend(batch["labels"].cpu().numpy())
        pred_labels_test.extend(predictions.cpu().numpy())

print("✅ Inference on Test Set complete.")

Evaluating on Test Set: 100%|██████████| 30/30 [00:02<00:00, 10.26it/s]

✅ Inference on Test Set complete.


In [ ]:
true_labels_test_flat = []
pred_labels_test_flat = []

for true_seq, pred_seq in zip(true_labels_test, pred_labels_test):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            true_labels_test_flat.append(id_to_label[true_label])
            pred_labels_test_flat.append(id_to_label[pred_label])

print("✅ Test labels flattened and aligned.")

✅ Test labels flattened and aligned.


In [ ]:
test_accuracy = accuracy_score(true_labels_test_flat, pred_labels_test_flat)
test_report = classification_report(true_labels_test_flat, pred_labels_test_flat, target_names=label_list, digits=4)

print(f"🔹 Test Accuracy: {test_accuracy:.4f}")
print("\n🔹 Test Classification Report:\n", test_report)

test_results_path = os.path.join(DATA_DIR, "RoBERTaTwitter_results", "test_results.txt")

with open(test_results_path, "w") as f:
    f.write(f"Test Accuracy: {test_accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(test_report)

print(f"✅ Test results saved to {test_results_path}")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🔹 Test Accuracy: 0.9699

🔹 Test Classification Report:
               precision    recall  f1-score   support

         SPA     0.7464    0.8427    0.7917       248
         ENG     0.8431    0.8053    0.8238       714
         BOR     0.8871    0.8045    0.8438      1504
         ENT     0.9959    0.9987    0.9973      3911
         OTH     0.0000    0.0000    0.0000         4
           N     0.9817    0.9914    0.9865     13468

    accuracy                         0.9699     19849
   macro avg     0.7424    0.7404    0.7405     19849
weighted avg     0.9692    0.9699    0.9693     19849

✅ Test results saved to /content/drive/My Drive/Capstone/RoBERTaTwitter_results/test_results.txt
